# Notebook demonstrating how to access GOES fire detection alerts data from geoserver

<img src='https://earthobservatory.nasa.gov/ContentFeature/GlobalFire/Images/goes_fire.jpg'><img src='https://upload.wikimedia.org/wikipedia/commons/6/63/GOES-R_SPACECRAFT.jpg' width='215'>

**What are Geostationary Operational Environmental Satellite (GOES) alerts and how do you query alerts data?**

The <a href = "https://en.wikipedia.org/wiki/Geostationary_Operational_Environmental_Satellite">Geostationary Operational Environmental Satellites (GOES)</a> support weather forecasting, severe storm tracking, meteorology research, and many other applications including wildfire detection.

<a href = "https://wifire-data.sdsc.edu/dataset/goes-fire-detections">GOES fire detection alerts data</a> shown in this notebook are generated from <a href = "https://en.wikipedia.org/wiki/GOES-16">GOES16</a> and <a href = "https://en.wikipedia.org/wiki/GOES-17">GOES17</a> satellites and accessed from <a href = "https://sdge.sdsc.edu/geoserver">geoserver </a>using the <a href = "https://docs.geoserver.org/stable/en/user/services/wfs/basics.html">Web Feature Service (WFS)</a> specification.

**References:**
    
- WFS: https://docs.geoserver.org/latest/en/user/services/wfs/reference.html
- cql filter: https://docs.geoserver.org/stable/en/user/tutorials/cql/cql_tutorial.html

<h3>Notebook Overview:</h3>
<ol><a href='#recent_alerts'><li>Query most recent GOES alert</li></a>
    <a href='#10recent_alerts'><li>Query 10 most recent alerts</li></a>
    <a href='#timestamps'><li>Query alerts between particular timestamps t1 and t2</li></a>
    <a href='#bbox'><li>Query alerts by bounding box</li></a>
    <a href='#detections'><li>Query alerts for only high quality detections</li></a>
    <a href='#plot'><li>Plot results on map and save data as shapefile</li></a>

In [ ]:
# folium installation recommended for map plotting visualizations
# !pip install folium

In [1]:
import requests, json, pprint, folium
import matplotlib.pyplot as plt
import geopandas as gpd

plt.style.use('ggplot')

# Here are all the features for GOES alerts

In [2]:
#let's take a look at a query geojson response in geopandas

baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = "service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&outputFormat=application%2Fjson" 

url = baseURL + selectionParameters

#url = 'https://sdge.sdsc.edu/geoserver/SDGE/ows?service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&outputFormat=application%2Fjson'

df = gpd.read_file(url)
df.head()

,id,filename,algorithm,sector,satellite,server_name,data_time,created_time,code,frp,...,obs_bt11,bkg_bt4,bkg_bt11,solar_zenith,satellite_zenith,relative_azimuth,ecosystem,fd_x,fd_y,geometry
0,1688982,f2022010200617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:06:17+00:00,2022-01-10T20:09:21.448000+00:00,12,223.8,...,286.033,290.869,277.277,54.236,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
1,1688983,f2022010201117.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:11:17+00:00,2022-01-10T20:14:30.535999+00:00,10,215.1,...,287.581,292.361,278.889,54.353,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
2,1688985,f2022010201617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:16:17+00:00,2022-01-10T20:19:22.195000+00:00,13,202.1,...,281.083,294.178,281.349,54.497,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
3,1688986,f2022010202117.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2022-01-10T20:21:17+00:00,2022-01-10T20:24:20.440001+00:00,10,138.0,...,287.090,295.841,282.407,54.671,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)
4,1688988,f2022010202617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:26:17+00:00,2022-01-10T20:29:19.580000+00:00,12,248.4,...,288.855,298.082,287.254,54.861,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)


<a id='recent_alerts'><h3>Query for the most recent GOES alert</h3></a>

#### This is a typical query reponse in geojson output

In [3]:
baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = "service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&count=1&sortBy=data_time&outputFormat=application%2Fjson" 

url = baseURL + selectionParameters

r = requests.get(url) 
data_dict = r.json()
data_dict


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'ssec_wfabba_goes.fid-7c6661d7_17e21ae3b98_7c55',
   'geometry': {'type': 'Point', 'coordinates': [-114.8316, 32.2031]},
   'geometry_name': 'geom',
   'properties': {'id': 1688982,
    'filename': 'f2022010200617.firelist.6_6_001g.FDCC.GOES-17.txt',
    'algorithm': '6_6_001g',
    'sector': 'FDCC',
    'satellite': 'GOES-17',
    'server_name': None,
    'data_time': '2022-01-10T20:06:17Z',
    'created_time': '2022-01-10T20:09:21.448Z',
    'code': 12,
    'frp': 223.8,
    'fire_size': -9999,
    'fire_temp': -9999,
    'line_num': 183,
    'element_num': 307,
    'pixel_size': 6144218,
    'obs_bt4': 326.887,
    'obs_bt11': 286.033,
    'bkg_bt4': 290.869,
    'bkg_bt11': 277.277,
    'solar_zenith': 54.236,
    'satellite_zenith': 44.812,
    'relative_azimuth': -9999,
    'ecosystem': 51,
    'fd_x': 3687,
    'fd_y': 1093}}],
 'totalFeatures': 29,
 'numberMatched': 29,
 'numberReturned': 1,
 'timeStamp': 

<a id='10recent_alerts'><h3>Query for the 10 most recent GOES alerts</h3></a>

In [4]:
baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = "service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&count=10&sortBy=data_time&outputFormat=application%2Fjson" 

url = baseURL + selectionParameters

#view in geopandas
most_recent_10_alerts = gpd.read_file(url)
most_recent_10_alerts
#most_recent_10['data_time']

# move forward with the geojson data

#r = requests.get(url) 
#data_dict = r.json()
#data_dict

,id,filename,algorithm,sector,satellite,server_name,data_time,created_time,code,frp,...,obs_bt11,bkg_bt4,bkg_bt11,solar_zenith,satellite_zenith,relative_azimuth,ecosystem,fd_x,fd_y,geometry
0,1688982,f2022010200617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:06:17+00:00,2022-01-10T20:09:21.448000+00:00,12,223.8,...,286.033,290.869,277.277,54.236,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
1,1688983,f2022010201117.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:11:17+00:00,2022-01-10T20:14:30.535999+00:00,10,215.1,...,287.581,292.361,278.889,54.353,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
2,1688985,f2022010201617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:16:17+00:00,2022-01-10T20:19:22.195000+00:00,13,202.1,...,281.083,294.178,281.349,54.497,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
3,1688991,f2022010202020.firelist.6_6_001g.FDCF.GOES-16.txt,6_6_001g,FDCF,GOES-16,None,2022-01-10T20:20:20+00:00,2022-01-10T20:30:34.738998+00:00,10,202.3,...,286.380,295.802,282.420,54.635,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)
4,1688990,f2022010202032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2022-01-10T20:20:32+00:00,2022-01-10T20:30:16.968000+00:00,12,233.3,...,282.870,295.863,283.382,54.630,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
5,1688986,f2022010202117.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2022-01-10T20:21:17+00:00,2022-01-10T20:24:20.440001+00:00,10,138.0,...,287.090,295.841,282.407,54.671,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)
6,1688989,f2022010202617.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2022-01-10T20:26:17+00:00,2022-01-10T20:29:21.340000+00:00,10,321.9,...,290.513,296.006,284.308,54.866,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)
7,1688988,f2022010202617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:26:17+00:00,2022-01-10T20:29:19.580000+00:00,12,248.4,...,288.855,298.082,287.254,54.861,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
8,1688993,f2022010211117.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T21:11:17+00:00,2022-01-10T21:14:21.100000+00:00,12,80.2,...,285.152,296.871,288.236,57.774,44.874,-9999,51,3686,1090,POINT (-114.83480 32.27390)
9,1689002,f2022010212020.firelist.6_6_001g.FDCF.GOES-16.txt,6_6_001g,FDCF,GOES-16,None,2022-01-10T21:20:20+00:00,2022-01-10T21:30:35.432999+00:00,10,235.1,...,293.836,293.688,288.920,58.538,56.945,-9999,51,1098,1127,POINT (-114.86420 32.25770)


<a id='timestamps'><h3>Query for GOES alerts between timestamps t1 and t2 using cql_filter</h3></a>

reference: https://docs.geoserver.org/stable/en/user/tutorials/cql/cql_tutorial.html

## let's choose a time window to query between

In [5]:
t1 = '2022-01-10T20:09:17Z'
t2 = '2022-01-10T21:06:17Z'

baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = f'service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&cql_filter=data_time%20BETWEEN%20{t1}AND%20{t2}&outputFormat=application%2Fjson'

url = baseURL + selectionParameters

#view in geopandas
time_window_query = gpd.read_file(url)
time_window_query
#most_recent_10['data_time']

# move forward with the geojson data

#r = requests.get(url) 
#data_dict = r.json()
#data_dict

,id,filename,algorithm,sector,satellite,server_name,data_time,created_time,code,frp,...,obs_bt11,bkg_bt4,bkg_bt11,solar_zenith,satellite_zenith,relative_azimuth,ecosystem,fd_x,fd_y,geometry
0,1688983,f2022010201117.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:11:17+00:00,2022-01-10T20:14:30.535999+00:00,10,215.1,...,287.581,292.361,278.889,54.353,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
1,1688985,f2022010201617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:16:17+00:00,2022-01-10T20:19:22.195000+00:00,13,202.1,...,281.083,294.178,281.349,54.497,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
2,1688991,f2022010202020.firelist.6_6_001g.FDCF.GOES-16.txt,6_6_001g,FDCF,GOES-16,None,2022-01-10T20:20:20+00:00,2022-01-10T20:30:34.738998+00:00,10,202.3,...,286.380,295.802,282.420,54.635,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)
3,1688990,f2022010202032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2022-01-10T20:20:32+00:00,2022-01-10T20:30:16.968000+00:00,12,233.3,...,282.870,295.863,283.382,54.630,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
4,1688986,f2022010202117.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2022-01-10T20:21:17+00:00,2022-01-10T20:24:20.440001+00:00,10,138.0,...,287.090,295.841,282.407,54.671,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)
5,1688989,f2022010202617.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2022-01-10T20:26:17+00:00,2022-01-10T20:29:21.340000+00:00,10,321.9,...,290.513,296.006,284.308,54.866,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)
6,1688988,f2022010202617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:26:17+00:00,2022-01-10T20:29:19.580000+00:00,12,248.4,...,288.855,298.082,287.254,54.861,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)


<a id='detections'><h3>Query for only quality detections</h3></a>

The quality of the fire detections is indicated by the 'code' feature. For example, a value of 15 and 35 is considered a low probability.  Here are the codes:

Code: 
- 10 (30) - Processed Fire Pixel (Temporally filtered)
- 11 (31) - Saturated Fire Pixel (Temporally filtered)
- 12 (32) - Cloudy Fire Pixel (Temporally filtered)
- 13 (33) - High Probability Fire Pixel (Temporally filtered)
- 14 (34) - Medium Probability Fire Pixel (Temporally filtered)
- 15 (35) - Low Probability Fire Pixel (Temporally filtered)  

**Let's query for fire detections with a Cloudy Fire Pixel (code = 12)**

In [6]:
code_value='12'

baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = f'service=WFS&version=2.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&count=100&cql_filter=code={code_value}&outputFormat=application%2Fjson'

url = baseURL + selectionParameters

#view in geopandas
time_window_query = gpd.read_file(url)
time_window_query
#most_recent_10['data_time']

# move forward with the geojson data

#r = requests.get(url) 
#data_dict = r.json()
#data_dict

,id,filename,algorithm,sector,satellite,server_name,data_time,created_time,code,frp,...,obs_bt11,bkg_bt4,bkg_bt11,solar_zenith,satellite_zenith,relative_azimuth,ecosystem,fd_x,fd_y,geometry
0,1688982,f2022010200617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:06:17+00:00,2022-01-10T20:09:21.448000+00:00,12,223.8,...,286.033,290.869,277.277,54.236,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
1,1688988,f2022010202617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:26:17+00:00,2022-01-10T20:29:19.580000+00:00,12,248.4,...,288.855,298.082,287.254,54.861,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
2,1688990,f2022010202032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2022-01-10T20:20:32+00:00,2022-01-10T20:30:16.968000+00:00,12,233.3,...,282.870,295.863,283.382,54.630,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
3,1688993,f2022010211117.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T21:11:17+00:00,2022-01-10T21:14:21.100000+00:00,12,80.2,...,285.152,296.871,288.236,57.774,44.874,-9999,51,3686,1090,POINT (-114.83480 32.27390)
4,1689003,f2022010213117.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T21:31:17+00:00,2022-01-10T21:34:19.153999+00:00,12,134.8,...,283.669,294.789,285.837,59.620,44.874,-9999,51,3686,1090,POINT (-114.83480 32.27390)
5,1689005,f2022010213117.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2022-01-10T21:31:17+00:00,2022-01-10T21:34:22.516001+00:00,12,232.1,...,285.958,293.012,287.366,59.563,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)
6,1689009,f2022010213617.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2022-01-10T21:36:17+00:00,2022-01-10T21:39:19.690001+00:00,12,219.2,...,283.691,291.380,285.928,60.075,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)


<a id='bbox'><h3>Query alerts by bounding box</h3></a>

In [7]:
# define the lat long coordinates for the bounding box

min_Longitude, min_Latitude, max_Longitude, max_Latitude = -116.09596,31.66970,-114.336939,32.259914

baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = f'service=WFS&version=1.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&srsName=EPSG:4326&maxFeatures=100&BBOX={min_Longitude},{min_Latitude},{max_Longitude},{max_Latitude}&outputFormat=application%2Fjson'

url = baseURL + selectionParameters

# format as geojson data
r = requests.get(url) 
data_dict = r.json()

#view in geopandas
time_window_query = gpd.read_file(url)
time_window_query



,id,filename,algorithm,sector,satellite,server_name,data_time,created_time,code,frp,...,obs_bt11,bkg_bt4,bkg_bt11,solar_zenith,satellite_zenith,relative_azimuth,ecosystem,fd_x,fd_y,geometry
0,1688982,f2022010200617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:06:17+00:00,2022-01-10T20:09:21.448000+00:00,12,223.8,...,286.033,290.869,277.277,54.236,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
1,1688983,f2022010201117.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:11:17+00:00,2022-01-10T20:14:30.535999+00:00,10,215.1,...,287.581,292.361,278.889,54.353,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
2,1688985,f2022010201617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:16:17+00:00,2022-01-10T20:19:22.195000+00:00,13,202.1,...,281.083,294.178,281.349,54.497,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
3,1688986,f2022010202117.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2022-01-10T20:21:17+00:00,2022-01-10T20:24:20.440001+00:00,10,138.0,...,287.090,295.841,282.407,54.671,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)
4,1688988,f2022010202617.firelist.6_6_001g.FDCC.GOES-17.txt,6_6_001g,FDCC,GOES-17,None,2022-01-10T20:26:17+00:00,2022-01-10T20:29:19.580000+00:00,12,248.4,...,288.855,298.082,287.254,54.861,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
5,1688989,f2022010202617.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2022-01-10T20:26:17+00:00,2022-01-10T20:29:21.340000+00:00,10,321.9,...,290.513,296.006,284.308,54.866,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)
6,1688990,f2022010202032.firelist.6_6_001g.FDCF.GOES-17.txt,6_6_001g,FDCF,GOES-17,None,2022-01-10T20:20:32+00:00,2022-01-10T20:30:16.968000+00:00,12,233.3,...,282.870,295.863,283.382,54.630,44.812,-9999,51,3687,1093,POINT (-114.83160 32.20310)
7,1688991,f2022010202020.firelist.6_6_001g.FDCF.GOES-16.txt,6_6_001g,FDCF,GOES-16,None,2022-01-10T20:20:20+00:00,2022-01-10T20:30:34.738998+00:00,10,202.3,...,286.380,295.802,282.420,54.635,56.890,-9999,51,1098,1129,POINT (-114.83240 32.20800)
8,1688999,f2022010212117.firelist.6_6_001g.FDCC.GOES-16.txt,6_6_001g,FDCC,GOES-16,None,2022-01-10T21:21:17+00:00,2022-01-10T21:24:21.497999+00:00,10,210.9,...,293.441,294.049,285.860,58.630,56.945,-9999,51,1098,1127,POINT (-114.86420 32.25770)
9,1689002,f2022010212020.firelist.6_6_001g.FDCF.GOES-16.txt,6_6_001g,FDCF,GOES-16,None,2022-01-10T21:20:20+00:00,2022-01-10T21:30:35.432999+00:00,10,235.1,...,293.836,293.688,288.920,58.538,56.945,-9999,51,1098,1127,POINT (-114.86420 32.25770)


<a id='plot'><h3>Plot results on map and demonstrate how to save data as shapefile</h3></a>

In [8]:
#Define coordinates of where we want to center our map
alert_coords = [32.20800,-114.83240]

#Create the map
my_map = folium.Map(location = alert_coords)

#GeoJSON alert marker overlays
folium.GeoJson(data_dict, name="geojson").add_to(my_map)   

#toggle layers
folium.LayerControl().add_to(my_map)  

#Display the map
my_map

#### save as shape file

In [9]:
#to save as Shapefile, change outputFormat=application%2Fjson to outputFormat=SHAPE-ZIP at the end of the url string

min_Longitude, min_Latitude, max_Longitude, max_Latitude = -116.09596,31.66970,-114.336939,32.259914

baseURL='https://sdge.sdsc.edu/geoserver/SDGE/ows?'

selectionParameters = f'service=WFS&version=1.0.0&request=GetFeature&typeName=SDGE:ssec_wfabba_goes&srsName=EPSG:4326&maxFeatures=100&BBOX={min_Longitude},{min_Latitude},{max_Longitude},{max_Latitude}&outputFormat=SHAPE-ZIP'

url = baseURL + selectionParameters

r = requests.get(url) 

with open("ssec_wfabba_goes.zip", "wb") as f:
    f.write(r.content)